In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [4]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [8]:
from pykinect2 import PyKinectV2
from pykinect2.PyKinectV2 import *
from pykinect2 import PyKinectRuntime
import numpy as np
import cv2
import ctypes
import _ctypes
import sys
import face_recognition
import os

kinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Color | PyKinectV2.FrameSourceTypes_Depth | PyKinectV2.FrameSourceTypes_Body)
    
known_face_encodings = []
known_face_names = []

# Load sample pictures and learn how to recognize it.
dirname = 'face_recognition/knowns'
files = os.listdir(dirname)
for filename in files:
    name, ext = os.path.splitext(filename)
    if ext == '.jpg':
        known_face_names.append(name)
        pathname = os.path.join(dirname, filename)
        img = face_recognition.load_image_file(pathname)
        face_encoding = face_recognition.face_encodings(img)[0]
        known_face_encodings.append(face_encoding)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
        
        # --- Getting frames and drawing
        if kinect.has_new_color_frame() & kinect.has_new_depth_frame():
            image_np = kinect.get_last_color_frame()
            image_np = np.reshape(image_np, (kinect.color_frame_desc.Height, kinect.color_frame_desc.Width, 4))
            frameC = kinect._color_frame_data
            image_np = cv2.cvtColor(image_np, cv2.COLOR_RGBA2RGB)
            
            # ----- Face Recognition
            # Resize frame of video to 1/4 size for faster face recognition processing
            rgb_small_frame = cv2.resize(image_np, (0, 0), fx=0.25, fy=0.25)

            # Only process every other frame of video to save time
            if process_this_frame:
                # Find all the faces and face encodings in the current frame of video
                face_locations = face_recognition.face_locations(rgb_small_frame)
                face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

                face_names = []
                for face_encoding in face_encodings:
                    # See if the face is a match for the known face(s)
                    distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                    min_value = min(distances)

                    # tolerance: How much distance between faces to consider it a match. Lower is more strict.
                    # 0.6 is typical best performance.
                    name = "Unknown"
                    if min_value < 0.6:
                        index = np.argmin(distances)
                        name = known_face_names[index]

                    face_names.append(name)

            process_this_frame = not process_this_frame
            
            # Display the results
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                # Scale back up face locations since the frame we detected in was scaled to 1/4 size
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                # Draw a box around the face
                cv2.rectangle(image_np, (left, top), (right, bottom), (0, 0, 255), 2)
                # Draw a label with a name below the face
                cv2.rectangle(image_np, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(image_np, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            
            
            # ----- Object Recognition
            # Definite input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            
            # Get coordinates of detected boxes ; ymin, ymax, xmin, xmax
            coordinates = vis_util.return_coordinates(
                        image_np,
                        np.squeeze(boxes),
                        np.squeeze(classes).astype(np.int32),
                        np.squeeze(scores),
                        category_index,
                        use_normalized_coordinates=True,
                        line_thickness=8,
                        min_score_thresh=0.60)
                        
            if coordinates is not None:
            
                # ----- Face Recognition
                for i in range(len(coordinates)):
                    if coordinates[i][0] == 'person':
                        coordinates[i][0] = name
                        
                #print(*coordinates)
                print(coordinates)
                xpoint_for_depth=[]
                ypoint_for_depth=[]
                
                print("object 개수:", len(coordinates))
                
                # Connect Color and Depth Info.
                frame = kinect.get_last_depth_frame()
                frameD = kinect._depth_frame_data
                frame = frame.astype(np.uint8)
                frame = np.reshape(frame, (424, 512))
                frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)

                # Take just one point : middle
                '''for i in range(len(coordinates)):

                    ypoint_for_depth.append(int((coordinates[i][1]+coordinates[i][2])/2))
                    xpoint_for_depth.append(int((coordinates[i][3]+coordinates[i][4])/2))
                    
                print("ypoint_for_depth:", ypoint_for_depth)
                print("xpoint_for_depth:", xpoint_for_depth)
                
                ##output = cv2.bilateralFilter(output, 1, 150, 75)
                cv2.imshow('KINECT Video Stream', frame)
            
                for i in range(len(coordinates)):
                    Pixel_Depth = frameD[((ypoint_for_depth[i] * 512) + xpoint_for_depth[i])]
                    print("점 하나만 고려:",i+1, "번째 object의 depth value: ", Pixel_Depth)'''
                    
                # Get average depth value considering all coordinate pairs. (List Version)
                '''Pixel_Depth_List = []
                Pixel_Depth_Sum = 0
                non_error_count = 0
                for i in range(len(coordinates)):
                    for x in range(int(coordinates[i][3]), int(coordinates[i][4]+1)):
                        for y in range(int(coordinates[i][1]), int(coordinates[i][2]+1)):
                            Pixel_Depth_Sum += frameD[((y * 512) + x)]
                            if frameD[((y * 512) + x)] != 0:
                                non_error_count += 1
                    
                    Pixel_Depth_List.append(int(Pixel_Depth_Sum/non_error_count))
                    
                for i in range(len(coordinates)):
                    print("전체 면적 고려:", i+1, "번째 object의 depth value: ", Pixel_Depth_List[i])
                print("\n")'''
                
                # Get average depth value considering all coordinate pairs. (Dict Version)
                Pixel_Depth_Dict = {}
                Pixel_Depth_Sum = 0
                non_error_count = 0
                for i in range(len(coordinates)):
                        
                    # ----- Depth Recognition
                    for x in range(int(coordinates[i][3]), int(coordinates[i][4]+1)):
                        for y in range(int(coordinates[i][1]), int(coordinates[i][2]+1)):
                            Pixel_Depth_Sum += frameD[((y * 512) + x)]
                            if frameD[((y * 512) + x)] != 0:
                                non_error_count += 1
                    
                    Pixel_Depth_Dict[coordinates[i][0]] = int(Pixel_Depth_Sum/non_error_count)
                
                # Compare each objects' depth. (Sorting: close -> far)
                sorted(Pixel_Depth_Dict.items(), key=lambda t : t[1])
                print(Pixel_Depth_Dict)
                print("\n")
                
                # VideoFileOutput.write(image_np)
                cv2.imshow('KINECT Color', image_np)
                cv2.waitKey(1)
                

        key = cv2.waitKey(1)
        if key == 27: break


[['chair', 612, 1074, 853, 1217, 84.20440554618835]]
object 개수: 1
{'chair': 25851}


[['chair', 609, 1074, 866, 1222, 85.52924394607544]]
object 개수: 1
{'chair': 25891}


[['chair', 606, 1074, 864, 1219, 82.34338164329529]]
object 개수: 1
{'chair': 25927}


[['chair', 607, 1074, 864, 1219, 85.45388579368591]]
object 개수: 1
{'chair': 25913}


[['chair', 610, 1073, 845, 1222, 87.18871474266052]]
object 개수: 1
{'chair': 25756}


[['chair', 609, 1071, 810, 1213, 78.46671342849731], ['SoraRyu', 0, 1070, 7, 874, 65.34036993980408]]
object 개수: 2
{'chair': 25798, 'SoraRyu': 16250}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[['chair', 604, 1072, 811, 1206, 62.831687927246094]]
object 개수: 1
{'chair': 25809}


[['chair', 609, 1072, 808, 1201, 60.162353515625]]
object 개수: 1
{'chair': 25837}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[['chair', 608, 1073, 807, 1205, 63.210463523864746]]
object 개수: 1
{'chair': 25542}


[['chair', 610, 1072,

{'SoraRyu': 13366, 'chair': 15636}


[['SoraRyu', 54, 1054, 8, 768, 76.11114978790283], ['chair', 565, 1037, 964, 1425, 74.58531856536865]]
object 개수: 2
{'SoraRyu': 13434, 'chair': 15731}


[['SoraRyu', 51, 1055, 10, 768, 79.00276780128479], ['chair', 567, 1037, 963, 1433, 66.49301052093506]]
object 개수: 2
{'SoraRyu': 14300, 'chair': 16592}


[['SoraRyu', 30, 1040, 8, 759, 78.98837327957153], ['chair', 559, 1039, 966, 1413, 68.70048642158508]]
object 개수: 2
{'SoraRyu': 13658, 'chair': 16002}


[['chair', 558, 1039, 964, 1408, 73.70792627334595], ['SoraRyu', 42, 1054, 0, 742, 62.40423917770386]]
object 개수: 2
{'chair': 25319, 'SoraRyu': 16251}


[['SoraRyu', 31, 1060, 1, 747, 80.47357201576233], ['chair', 563, 1037, 971, 1425, 79.15916442871094]]
object 개수: 2
{'SoraRyu': 14033, 'chair': 16263}


[['SoraRyu', 36, 1037, 9, 759, 86.95744276046753], ['chair', 560, 1037, 969, 1432, 70.2796459197998]]
object 개수: 2
{'SoraRyu': 13573, 'chair': 15973}


[['SoraRyu', 35, 1051, 5, 752, 82.27400779724

[['chair', 563, 1038, 966, 1433, 69.05690431594849]]
object 개수: 1
{'chair': 25255}


[['chair', 563, 1038, 965, 1420, 69.29882764816284]]
object 개수: 1
{'chair': 25483}


[['chair', 562, 1037, 966, 1421, 67.70300269126892], ['SoraRyu', 60, 1050, 10, 747, 65.37455320358276]]
object 개수: 2
{'chair': 25515, 'SoraRyu': 16725}


[['SoraRyu', 55, 1054, 6, 758, 79.17428016662598], ['chair', 564, 1039, 966, 1434, 66.55676364898682]]
object 개수: 2
{'SoraRyu': 14317, 'chair': 16578}


[['SoraRyu', 49, 1052, 6, 758, 78.19396257400513], ['chair', 563, 1037, 967, 1418, 77.40156650543213]]
object 개수: 2
{'SoraRyu': 14159, 'chair': 16379}


[['SoraRyu', 43, 1058, 5, 770, 80.93522787094116], ['chair', 564, 1039, 969, 1435, 74.94008541107178]]
object 개수: 2
{'SoraRyu': 14220, 'chair': 16481}


[['SoraRyu', 42, 1048, 5, 762, 81.9326639175415], ['chair', 562, 1037, 973, 1444, 68.61645579338074]]
object 개수: 2
{'SoraRyu': 13882, 'chair': 16232}


[['SoraRyu', 46, 1048, 8, 760, 84.45819616317749], ['chair', 565,

{'SoraRyu': 13746, 'chair': 16077}


[['SoraRyu', 103, 1052, 17, 767, 87.29267120361328], ['chair', 566, 1038, 965, 1440, 67.3477053642273]]
object 개수: 2
{'SoraRyu': 14883, 'chair': 17140}


[['SoraRyu', 224, 1054, 13, 774, 79.38414812088013], ['chair', 562, 1037, 965, 1437, 79.25128936767578]]
object 개수: 2
{'SoraRyu': 18727, 'chair': 20539}


[['chair', 563, 1036, 965, 1424, 78.66607308387756], ['SoraRyu', 232, 1061, 8, 779, 78.48442792892456]]
object 개수: 2
{'chair': 25733, 'SoraRyu': 20820}


[['SoraRyu', 224, 1057, 9, 782, 82.35312104225159], ['chair', 565, 1038, 967, 1435, 77.57638096809387]]
object 개수: 2
{'SoraRyu': 18656, 'chair': 20380}


[['SoraRyu', 224, 1065, 12, 778, 81.78520202636719], ['chair', 568, 1037, 972, 1423, 78.46784591674805]]
object 개수: 2
{'SoraRyu': 18865, 'chair': 20506}


[['SoraRyu', 223, 1068, 9, 781, 83.02276134490967], ['chair', 564, 1037, 969, 1430, 77.15524435043335]]
object 개수: 2
{'SoraRyu': 18920, 'chair': 20574}


[['chair', 565, 1037, 970, 1423, 80.2

[['backpack', 427, 1052, 405, 959, 76.60685181617737]]
object 개수: 1
{'backpack': 25133}


[['backpack', 434, 1050, 409, 965, 77.38094329833984]]
object 개수: 1
{'backpack': 25144}


[['backpack', 422, 1046, 393, 968, 78.3878743648529]]
object 개수: 1
{'backpack': 25336}


[['backpack', 433, 1048, 401, 950, 60.48709750175476]]
object 개수: 1
{'backpack': 25344}


[]
object 개수: 0
{}


[['backpack', 428, 1051, 405, 959, 60.89140772819519]]
object 개수: 1
{'backpack': 25238}


[['backpack', 418, 1049, 390, 971, 63.318657875061035]]
object 개수: 1
{'backpack': 25057}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[['SoraRyu', 449, 1044, 421, 980, 83.072429895401]]
object 개수: 1
{'SoraRyu': 25335}


[['SoraRyu', 451, 1041, 423, 975, 75.79795122146606]]
object 개수: 1
{'SoraRyu': 25341}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[['SoraRyu', 440, 1049, 414, 1028, 82.69556760787964], ['chair', 568, 1033, 1063, 1609, 68.844586

...

[['chair', 564, 1033, 957, 1461, 73.93710017204285]]
object 개수: 1
{'chair': 25342}


[['chair', 567, 1034, 967, 1467, 71.93441390991211]]
object 개수: 1
{'chair': 25329}


[['chair', 564, 1036, 938, 1459, 73.67825508117676]]
object 개수: 1
{'chair': 25475}


[['chair', 567, 1035, 940, 1457, 73.68074059486389]]
object 개수: 1
{'chair': 25423}


[['chair', 562, 1035, 937, 1458, 73.60312342643738]]
object 개수: 1
{'chair': 25442}


[['chair', 564, 1034, 931, 1456, 70.8436369895935]]
object 개수: 1
{'chair': 25455}


[['chair', 565, 1033, 944, 1458, 72.30958342552185]]
object 개수: 1
{'chair': 25370}


[['chair', 568, 1036, 932, 1459, 75.7311999797821]]
object 개수: 1
{'chair': 25191}


[['chair', 567, 1033, 943, 1464, 76.17632150650024], ['SoraRyu', 454, 1048, 555, 936, 63.70704174041748]]
object 개수: 2
{'chair': 25165, 'SoraRyu': 25189}


[['chair', 568, 1029, 945, 1445, 75.99351406097412], ['SoraRyu', 465, 1039, 586, 932, 72.19193577766418]]
object 개수: 2
{'chair': 25248, 'SoraRyu': 25317}


[['chair',

{'chair': 24868, 'SoraRyu': 24564}


[['chair', 555, 1020, 1037, 1505, 70.27546763420105], ['SoraRyu', 372, 1030, 349, 812, 68.41719150543213]]
object 개수: 2
{'chair': 24922, 'SoraRyu': 24731}


[['chair', 552, 1021, 1003, 1491, 67.31510162353516], ['chair', 537, 1048, 732, 1496, 61.3319456577301]]
object 개수: 2
{'chair': 25296}


[['chair', 551, 1021, 1012, 1492, 61.33652925491333]]
object 개수: 1
{'chair': 25100}


[['chair', 554, 1021, 1035, 1499, 64.59495425224304], ['chair', 544, 1038, 742, 1453, 60.9245240688324]]
object 개수: 2
{'chair': 25301}


[['chair', 551, 1019, 1033, 1502, 71.32835388183594], ['chair', 544, 1041, 758, 1452, 66.15719795227051]]
object 개수: 2
{'chair': 25283}


[['chair', 542, 1041, 745, 1461, 68.115895986557], ['chair', 551, 1018, 1031, 1497, 67.84895062446594]]
object 개수: 2
{'chair': 25071}


[['chair', 535, 1043, 746, 1486, 73.87189269065857], ['chair', 552, 1022, 1020, 1503, 70.52160501480103]]
object 개수: 2
{'chair': 24997}


[['chair', 550, 1018, 1023, 1501, 

{'chair': 25200}


[]
object 개수: 0
{}


[['chair', 555, 1046, 660, 1361, 66.68180227279663]]
object 개수: 1
{'chair': 25360}


[['SoraRyu', 410, 1039, 354, 1421, 73.36534261703491]]
object 개수: 1
{'SoraRyu': 25207}


[['chair', 544, 1041, 671, 1333, 60.402947664260864]]
object 개수: 1
{'chair': 25428}


[['SoraRyu', 385, 1049, 179, 1452, 61.27830147743225]]
object 개수: 1
{'SoraRyu': 25003}


[['SoraRyu', 385, 1055, 308, 1372, 64.17444944381714]]
object 개수: 1
{'SoraRyu': 25024}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[['SoraRyu', 342, 1056, 264, 1421, 70.97642421722412]]
object 개수: 1
{'SoraRyu': 23935}


[['SoraRyu', 373, 1052, 208, 1418, 66.15225076675415]]
object 개수: 1
{'SoraRyu': 24631}


[['SoraRyu', 372, 1052, 303, 1380, 78.21551561355591]]
object 개수: 1
{'S

{'SoraRyu': 19738}


[['SoraRyu', 293, 1031, 283, 721, 78.94980907440186]]
object 개수: 1
{'SoraRyu': 20926}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[['chair', 540, 1042, 622, 1316, 63.76727819442749]]
object 개수: 1
{'chair': 25214}


[['chair', 537, 1042, 614, 1325, 63.6867880821228]]
object 개수: 1
{'chair': 25385}


[]
object 개수: 0
{}


[['chair', 535, 1041, 605, 1354, 63.891589641571045]]
object 개수: 1
{'chair': 25348}


[['chair', 538, 1040, 609, 1325, 65.64105153083801]]
object 개수: 1
{'chair': 25328}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
ob

KeyboardInterrupt: 